IMPORTS

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
from pprint import pprint
import time

from scipy.stats import linregress
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# Import API key
from api_key import api_key



# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data_cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

CITY LIST

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

598

In [3]:
lat_lng

(-75.70992373069033, -151.78327096466657)

In [4]:
cities_sub = cities[0:10]
cities_sub

['dingle',
 'puerto madryn',
 'hithadhoo',
 'klaksvik',
 'albany',
 'castro',
 'wulanhaote',
 'san juan del cesar',
 'mataura',
 'belmonte']

# Make Request

In [5]:
lats = []
longs = []
temps = []
humids = []
clouds = []
speeds = []
cities_found = []
cloudiness = []

for i in range(len(cities_sub)):
    city = cities_sub[i]

for city in cities:
    #create the url
    units = "imperial"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units={units}&appid={api_key}"
#    print(url)

    #make the request
    response = requests.get(url)
#    print(response.status_code)
    
    #error check
    if response.status_code == 200:
        data = response.json()
       # pprint(data)
    
        try:        
            #extract the data
            lat = data["coord"]["lat"]
            long = data["coord"]["lon"]
            temp = data["main"]["temp"]
            feels_like = data["main"]["feels_like"]
            humidity = data["main"]["humidity"]        
            speed = data["wind"]["speed"]
            clouds = data["clouds"]["all"]
            city_name = data["name"]
        
            #save the data
            lats.append(lat)
            longs.append(long)        
            temps.append(temp)        
            humids.append(humidity)       
            speeds.append(speed)
            cities_found.append(city_name)
            cloudiness.append(clouds)
        except Exception as e:
            print(f"Through exception for city {city}: {e}")
        
    elif response.status_code == 404:
        print(f"Missing data in Open/WeatherAPI for {city}")
    else:
        print(response.status_code)
        print("Oh gosh darnit. The API is broken. Sad Face.")
    
    #Print every 5 cities
    if (i % 5 == 0):
        print(f"Data for city index: {i} of {len(cities)}")
    
    time.sleep(1)
        
# make the dataframe
df = pd.DataFrame()
df["City"] = cities_found
df["Latitude"] = lats
df["Longitude"] = longs
df["Temperature"] = temps
df["Humidity"] = humids
df["Cloudiness"] = clouds
df["Wind Speed"] = speed

df.head(10)

Missing data in Open/WeatherAPI for wulanhaote
Missing data in Open/WeatherAPI for nizhneyansk
Missing data in Open/WeatherAPI for sumbawa
Missing data in Open/WeatherAPI for samusu
Missing data in Open/WeatherAPI for barentsburg
Missing data in Open/WeatherAPI for buqayq
Missing data in Open/WeatherAPI for bur gabo
Missing data in Open/WeatherAPI for taolanaro
Missing data in Open/WeatherAPI for amderma
Missing data in Open/WeatherAPI for tunduru
Missing data in Open/WeatherAPI for bolshoye soldatskoye
Missing data in Open/WeatherAPI for grand river south east
Missing data in Open/WeatherAPI for attawapiskat
Missing data in Open/WeatherAPI for marcona
Missing data in Open/WeatherAPI for andarab
Missing data in Open/WeatherAPI for tabiauea
Missing data in Open/WeatherAPI for illoqqortoormiut
Missing data in Open/WeatherAPI for atka
Missing data in Open/WeatherAPI for cumaribo
Missing data in Open/WeatherAPI for belushya guba
Missing data in Open/WeatherAPI for tsihombe
Missing data in 

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Dingle,11.00,122.67,83.41,77,0,5.82
1,Puerto Madryn,-42.77,-65.04,66.79,47,0,5.82
2,Hithadhoo,-0.60,73.08,81.82,79,0,5.82
3,Klaksvík,62.23,-6.59,50.00,93,0,5.82
4,Albany,42.60,-73.97,42.66,71,0,5.82
5,Castro,-24.79,-50.01,62.44,91,0,5.82
6,San Juan del Cesar,10.77,-73.00,74.08,92,0,5.82
7,Mataura,-46.19,168.86,76.96,21,0,5.82
8,Belmonte,-15.86,-38.88,77.00,73,0,5.82
9,Ilulissat,69.22,-51.10,24.80,68,0,5.82


In [6]:
df.to_csv(output_data_file)